In [2]:
#import necessary libraries
from ultralytics import YOLO
import torch
from PIL import Image
import cv2
from segment_anything import SamPredictor, sam_model_registry
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import supervision as sv
from scipy.stats import gaussian_kde
from sklearn.model_selection import StratifiedShuffleSplit
import plotly.express as px
import mplcursors
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [3]:
# define mask and box functions
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [ ]:
# load the custom YOLO model
weight = #path to custom YOLO object detection model
model = YOLO(weight)

# run the model on a single image
results = model.predict(#path to image, conf=0.5)

# extract bounding box coordinates from the detection
for result in results:
    boxes = result.boxes
bbox = boxes.xyxy.tolist()[0]
device = "cuda"

# run the segmentation model on the bounding box
image = cv2.cvtColor(cv2.imread(#path to image), cv2.COLOR_BGR2RGB)
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)
predictor.set_image(image)
input_box = np.array(bbox)
masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False,
)

# save the output image with the segmentation mask as a variable
output_image = image
output_mask = masks[0]

# get the segmentation mask
segmentation_mask = masks[0]

# convert the segmentation mask to a binary mask
binary_mask = np.where(segmentation_mask > 0.5, 1, 0)

# create a background with NA values
na_background = np.full_like(image, np.nan)

# apply the binary mask
img_na = na_background * (1 - binary_mask[..., np.newaxis]) + image * binary_mask[..., np.newaxis]

# set the red, green, and blue channels as variables
s_r = img_na[:,:,0]
s_g = img_na[:,:,1]
s_b = img_na[:,:,2]

# performing the red/green ratio calculation
rg = s_r / (s_g + s_r)

# set the pixels on rg that are not within the mask to NaN
rg[~segmentation_mask] = np.nan

In [5]:
# activate interactive mode
%matplotlib widget

In [ ]:
# Visualize the image segmentation mask and the rg transformation

# Set the plot style to a dark theme
plt.style.use('classic')

# create a new figure and arrange the histogram and exgr image side by side with a colorbar, with the original image and the image with the segmentation mask above
plt.figure(figsize=(15, 15))

# Plot the original image on the top left
plt.subplot(2, 2, 1)
plt.imshow(image)
plt.title('Original Image')
plt.axis('off')

# Plot the image with the segmentation mask on the top right
plt.subplot(2, 2, 2)
plt.imshow(image)
plt.title('Segmentation Mask')
show_mask(masks[0], plt.gca())
show_box(input_box, plt.gca())
plt.axis('off')

# Plot the exgr image on the bottom left
plt.subplot(2, 2, 3)
plt.imshow(rg, cmap='Spectral')
plt.colorbar()
plt.title('RG Transformation')

# Filter out invalid values (inf and NaN) from the exgr array
valid_rg = rg.ravel()[np.isfinite(rg.ravel())]

# Plot the smoothed density histogram on the bottom right
plt.subplot(2, 2, 4)

# Compute the smoothed density histogram using the kernel density estimate (KDE)
kde = gaussian_kde(valid_rg)
x_vals = np.linspace(np.min(valid_rg), np.max(valid_rg), 500)  # Values for the x-axis

# Obtain colors from the 'viridis' colormap for the histogram bars
colors = plt.get_cmap('Spectral')(np.linspace(0, 1, len(x_vals)))

# Plot the histogram as bars with colors based on x-value
plt.bar(x_vals, kde(x_vals), width=(x_vals[1]-x_vals[0]), color=colors, edgecolor='none')

# Plot a line graph on top of the histogram
plt.plot(x_vals, kde(x_vals), color='black')

# Add a vertical line at x=0.49
plt.axvline(x=0.49, color='black', linestyle='--') #change this value to the threshold value

# Add labels and title
plt.xlabel('RG Values')
plt.ylabel('Density')
plt.title('Smoothed Density Histogram')

# Show the plot
plt.show()

In [38]:
# create a function to count the number of infected pixels
def count_pixels_below_threshold(rg):
    # Flatten the exgr image into a 1D array
    rg_flat = rg.ravel()

    # Count the number of pixels below threshold
    count = np.sum(rg_flat > 0.49) # change this value to the threshold value

    return count

# create a function to count the total number of pixels in the head
def count_non_na_pixels(rg):
    # Flatten the exgr image into a 1D array
    rg_flat = rg.ravel()

    # Count the number of non-NA pixels
    count = np.sum(np.isfinite(rg_flat))

    return count

# count the total number of pixels below the threshold
num_pixels_below_threshold = count_pixels_below_threshold(rg)

# count the total number of pixels in the head
num_non_na_pixels = count_non_na_pixels(rg)

# calculate the proportion of pixels below the threshold
sum = (num_pixels_below_threshold / num_non_na_pixels)*100
print('The proportion of pixels below the threshold is', sum, '%')

In [ ]:
# determine the accuracy of the rg transformation severity estimations through stratified random sampling

# Set the pixels within the mask to binary based on the rg value
binary_mask = np.where((segmentation_mask > 0) & (rg > 0.49), 1, 0)

# visualize the binary mask
plt.imshow(binary_mask, cmap='gray')
plt.show()

# Get the indices of all non-zero pixels in the binary mask
indices = np.argwhere(segmentation_mask > 0)

# Get the values of each pixel in the binary mask
values = binary_mask[indices[:, 0], indices[:, 1]]

# Perform stratified random sampling to select 30 indices with a proportional amount of 0s and 1s
sss = StratifiedShuffleSplit(n_splits=1, test_size=30, random_state=42)
train_index, test_index = next(sss.split(indices, values))

# Get the selected indices and their corresponding values
selected_indices = indices[test_index]
selected_values = values[test_index]

# Create a DataFrame to store the position and value of each selected pixel
df = pd.DataFrame(columns=['Row', 'Column', 'Value'])

# Loop through each selected index and record its position and value in a list of dictionaries
data = []
for index, value in zip(selected_indices, selected_values):
    row, col = index
    data.append({'Row': row, 'Column': col, 'Value': value})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data, columns=['Row', 'Column', 'Value'])

# Print the DataFrame to the console
print(df)

In [ ]:
#visualize selected pixels on the image

# Load the original RGB image
img = cv2.imread(#path to image)

# Set the row and column numbers for the desired pixel
row_num = #enter row number
col_num = #enter column number

# Highlight the desired pixel in the image
img_highlighted = img.copy()
cv2.circle(img_highlighted, (col_num, row_num), 5, (0, 0, 255), -1)

# Display the highlighted image using Matplotlib
plt.imshow(cv2.cvtColor(img_highlighted, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# assess the accuracy of the rg transformation severity estimations 

# Load the Excel file into a DataFrame
df = pd.read_excel(#path to Excel file with the estimated and observed severity scores)

# Get the observed and predicted class labels from the DataFrame
observed = df['Visual']
predicted = df['Value']

# Create a confusion matrix
cm = confusion_matrix(observed, predicted)

# Calculate accuracy, recall, precision, and F1 scores
accuracy = accuracy_score(observed, predicted)
recall = recall_score(observed, predicted, average='weighted')
precision = precision_score(observed, predicted, average='weighted')
f1 = f1_score(observed, predicted, average='weighted')

# Display the confusion matrix as a heatmap using Matplotlib 
sns.heatmap(cm, annot=True, cmap='Blues', annot_kws={'size': 15})
plt.xlabel('Predicted Class', fontsize=15)
plt.ylabel('Observed Class', fontsize=15)
plt.title('Confusion Matrix', fontsize=15)
plt.show()

# Print the scores to the console
print('Accuracy:', accuracy)
print('Recall:', recall)
print('Precision:', precision)
print('F1 Score:', f1)

In [ ]:
# evaulate severity over a folder of images

#set device to gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def count_below_threshold(rg, threshold):
    """
    Counts the number of pixels below the given threshold in the given exgr image.
    """
    # Flatten the exgr image into a 1D array
    rg_flat = rg.ravel()

    # Count the number of pixels below the threshold
    count = np.sum(rg_flat > threshold)

    return count

# set the path to the folder containing the images
folder_path = # path to folder containing images

# load the custom YOLO model
weight = # path to custom YOLO object detection model
model = YOLO(weight)

# initialize a list to store the ratios for each image
ratio = []
isolate = []
cultivar = []
rep = []
head = []
DAI = []
data = []

# loop over all the files in the folder
for filename in os.listdir(folder_path):
    # check if the file is an image
    if filename.endswith('.png') or filename.endswith('.jpg'):
         # extract isolate, cultivar, rep, and DAI information from the filename
        isolate, cultivar, rep, head, DAI = filename.split('_')[:5]
        # run the model on the image
        image_path = os.path.join(folder_path, filename)
        results = model.predict(image_path, conf=0.5)

        # extract bounding box coordinates from the detection
        for result in results:
            boxes = result.boxes
            if len(boxes) > 0:
                bbox = boxes.xyxy.tolist()[0]
                device = "cuda"

        # run the segmentation model on the bounding box
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        sam_checkpoint = "sam_vit_h_4b8939.pth"
        model_type = "vit_h"
        sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
        sam.to(device=device)
        predictor = SamPredictor(sam)
        predictor.set_image(image)
        input_box = np.array(bbox)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box[None, :],
            multimask_output=False,
        )
        # Extract information from the file name
        file_name = os.path.splitext(filename)[0]
        name_parts = file_name.split("_")
        isolate_name = name_parts[0]
        cultivar_name = name_parts[1]
        rep = name_parts[2]
        head_number = name_parts[3]
        days_after_infection = name_parts[4]

        #get the segmentation mask
        segmentation_mask = masks[0]

        # Convert the segmentation mask to a binary mask
        binary_mask = np.where(segmentation_mask > 0.5, 1, 0)

        #create a background with NA values
        na_background = np.full_like(image, np.nan)

        # Apply the binary mask
        img_na = na_background * (1 - binary_mask[..., np.newaxis]) + image * binary_mask[..., np.newaxis]

        s_r = img_na[:,:,0]
        s_g = img_na[:,:,1]
        s_b = img_na[:,:,2]

        #performing the red/green ratio calculation
        rg = s_r / (s_g + s_r)

        # set the pixels on exgr that are not within the mask to NaN
        rg[~segmentation_mask] = np.nan

        # Calculate the total number of non-NA pixels
        total_pixels = np.sum(np.isfinite(rg))

        # Calculate the number of pixels below the threshold
        threshold = 0.49 # change this value to the threshold value
        below_threshold = count_below_threshold(rg, threshold)

        # Calculate the ratio of pixels below the threshold to total pixels
        ratio = below_threshold / total_pixels

        image_info = {
                        'Isolate': isolate_name,
                        'Cultivar': cultivar_name,
                        'Replication': rep,
                        'Head': head_number,
                        'DAI': days_after_infection,
                        'Below Threshold': below_threshold,
                        'Total Pixels': total_pixels,
                        'Severity': ratio,
                        
                    }

        data.append(image_info)
        
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_csv(#path to savet csv file, index=False)